In [1]:
# Info about the variables
file = '2014-12-03_P005_Post_002'
frame = 90

In [2]:
import os

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')

import modules.plotting as pl
import modules.iterable_funcs as itf

In [3]:
%matplotlib auto

Using matplotlib backend: MacOSX


### Load variables

In [36]:
load_dir = os.path.join('data', 'saved_variables')

population = np.load(os.path.join(load_dir, 'population.npy'))
labels = np.load(os.path.join(load_dir, 'labels.npy'))
path_matrix = np.load(os.path.join(load_dir, 'path_matrix.npy'))

In [5]:
part_types = ['Head', 'Hip', 'Thigh', 'Knee', 'Calf', 'Foot']

### Plot population

In [34]:
pl.scatter_labels(population, labels)

plt.legend(part_types)
plt.xlim((-150, 0))
plt.xlabel('X')
plt.ylabel('Y', rotation=0)
plt.show()

### Plot spheres

In [50]:
path_0, path_1 = path_matrix[3, :], path_matrix[1, :]
pop_0, pop_1 = population[path_0, :], population[path_1, :]

pl.scatter_labels(population, labels)
pl.scatter2(pop_0, s=1e3, facecolors='none', edgecolors='k')
pl.scatter2(pop_1, s=1e3, facecolors='none', edgecolors='k')

plt.legend(part_types)
plt.xlim((-150, 0))
plt.xlabel('X')
plt.ylabel('Y', rotation=0)
plt.show()

### Plot body graph

In [7]:
def generate_points(n_points_per_set):
    
    n_sets = len(n_points_per_set)

    for i, n_points in enumerate(n_points_per_set):

        y_vals = (n_sets - i) * np.ones(n_points)
        x_vals = np.linspace(0, 1, n_points + 1, endpoint=False)[1:]
        
        points = np.stack([(x, y) for x, y in zip(x_vals, y_vals)])
        
        yield points

In [8]:
n_points_per_set = [2, 3, 5, 2, 4, 5]

point_sets = [*generate_points(n_points_per_set)]

In [17]:
plt.figure()

for points_a, points_b in itf.pairwise(point_sets):
        
    points = np.vstack([points_a, points_b])
    
    pl.scatter2(points, c='k', s=50)
    pl.connect_two_sets(points_a, points_b, c='k')    
    
    
y_coords = [points[0, 1] for points in point_sets]

for i, part_type in enumerate(part_types):
    
    plt.text(1, y_coords[i], part_type, fontsize=12)

plt.xlim((0, 1.1))
plt.axis('off')
plt.show()
